In [42]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_2/matryca_transformacja_1/data

/content/drive/My Drive/Colab Notebooks/matrix/matrix_2/matryca_transformacja_1/data


In [44]:
df = pd.read_hdf('car.h5')
df.shape

(106494, 155)

In [45]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

In [0]:
SUFFIX_CAT = '__cat'


for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [47]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision Tree


In [49]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [50]:
model = RandomForestRegressor(max_depth = 5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [51]:
xgb_params = {
    'max_depth' :5, 
    'n_estimators': 50, 
    'learning_rate' : 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[11:37:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:38:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [25]:
xgb_params = {
    'max_depth' :5, 
    'n_estimators': 50, 
    'learning_rate' : 0.1,
    'seed': 0
}
m = xgb.XGBRegressor(**xgb_params)
m.fit(x, y)
# Pozwala wyświetlić jak duży wpływ dana cecha ma na y danego modelu
imp = PermutationImportance(m, random_state=42).fit(x,y)
eli5.show_weights(imp, feature_names=cat_feats)

[11:05:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1197 ± 0.0028,param_napęd__cat
0.1134 ± 0.0014,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0614 ± 0.0007,param_skrzynia-biegów__cat
0.0576 ± 0.0017,param_faktura-vat__cat
0.0495 ± 0.0021,param_moc__cat
0.0276 ± 0.0002,param_marka-pojazdu__cat
0.0240 ± 0.0007,feature_kamera-cofania__cat
0.0210 ± 0.0006,param_typ__cat
0.0180 ± 0.0013,seller_name__cat


In [52]:
feats = ['param_napęd__cat', 'param_rok-produkcji','param_stan__cat' ,'param_skrzynia-biegów__cat','param_faktura-vat__cat',
'param_moc', 'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'seller_name__cat',
'param_pojemność-skokowa',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat',]
len(feats)

20

In [58]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[11:40:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:40:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:40:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9564.58895203383, 73.0539676664732)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x:  1 if str(x) == 'None' else int(x.split(' ')[0]))

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:  1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))